In [102]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 9 :
* Delete Duplicate
* Delete Null
* Outlier capping winsorize sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [103]:
df = pd.read_csv('../../Without Feature Selection/UFC_kombinasi9_all_features.csv')

In [104]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [105]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi9_all_features.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1, errors='ignore')
# Get the common columns between df and df_test
common_columns = list(set(X.columns) & set(df_test.columns))
# Update df_test to only include the common columns
df_test = df_test[common_columns]
X = X[common_columns]

In [106]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [107]:
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization
# from keras.callbacks import EarlyStopping, LearningRateScheduler
# from keras import backend as K
# from keras.regularizers import l1

# # Define R-squared metric
# def r_squared(y_true, y_pred):
#     SS_res =  K.sum(K.square(y_true - y_pred)) 
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
#     return (1 - SS_res/(SS_tot + K.epsilon()))

# # Define the model architecture
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(256, activation='relu', kernel_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu', kernel_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(1))

# # Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r_squared])

# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# # Define learning rate scheduler
# def scheduler(epoch, lr):
#   if epoch < 10:
#     return lr
#   else:
#     return lr * tf.math.exp(-0.1)

# lr_scheduler = LearningRateScheduler(scheduler)

# # Train the model
# model.fit(X, y, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])

# # Evaluate the model
# loss, r2_score = model.evaluate(X, y)
# print("R-squared:", r2_score)

In [108]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.regularizers import l1

# Define R-squared metric
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

# Define the model architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1],), activity_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu', activity_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', activity_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r_squared])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Train the model
model.fit(X, y, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, r2_score = model.evaluate(X, y)
print("R-squared:", r2_score)

Epoch 1/300
87/87 [==============================] - 1s 9ms/step - loss: 33342.7422 - r_squared: -429.0867 - val_loss: 33079.1328 - val_r_squared: -408.6971
Epoch 2/300
87/87 [==============================] - 0s 5ms/step - loss: 32407.5547 - r_squared: -404.6611 - val_loss: 31833.7617 - val_r_squared: -392.9194
Epoch 3/300
87/87 [==============================] - 0s 5ms/step - loss: 30305.1406 - r_squared: -387.2776 - val_loss: 29227.1504 - val_r_squared: -360.6837
Epoch 4/300
87/87 [==============================] - 1s 6ms/step - loss: 26712.5371 - r_squared: -332.6939 - val_loss: 24780.1406 - val_r_squared: -305.5268
Epoch 5/300
87/87 [==============================] - 0s 6ms/step - loss: 22011.0430 - r_squared: -276.2962 - val_loss: 19635.2031 - val_r_squared: -241.7576
Epoch 6/300
87/87 [==============================] - 0s 5ms/step - loss: 16796.4453 - r_squared: -208.2934 - val_loss: 14333.8682 - val_r_squared: -176.0924
Epoch 7/300
87/87 [==============================] - 1s 6m

In [109]:
df_test_scaled = scaler.transform(df_test)
y_pred = model.predict(df_test_scaled)

25/25 [==============================] - 0s 2ms/step


In [110]:
predictions_df = pd.DataFrame(y_pred, columns=['Predicted'])

In [111]:
predictions_df['id'] = df_test_id
submission = predictions_df[['id', 'Predicted']]

In [112]:
submission.head()

,id,Predicted
0,0.0,166.826508
1,1.0,169.806915
2,2.0,186.004547
3,3.0,169.545288
4,4.0,174.530151


In [113]:
submission.to_csv('pred_kombinasi3_neural_network_test.csv', index=False)